In [1]:
'''
Load json file, then extract info into a list
'''
import json

with open('mdpfiles\\predicted_output\\result.json') as json_file:

    data = json.load(json_file)
    
    all_images = []

    for each in data:
        name = each['objects'][0]['name']
        x_axis = each['objects'][0]['relative_coordinates']['left_x']
        y_axis = each['objects'][0]['relative_coordinates']['top_y']
        confidence = each['objects'][0]['confidence']
        image_info = [name, x_axis, y_axis, confidence]
        all_images.append(image_info)

In [2]:
'''
After extracting info into a list,
declare new df and set its values according to the list
'''

import pandas as pd
import numpy as np

columns = [
    'Description',
    'BBox X-axis',
    'BBox Y-axis',
    'Confidence'
]

df = pd.DataFrame(data = all_images, columns = columns) 

In [3]:
'''
Import helper function to get each respective image's path
'''

from filelister import *

dir_name = 'mdpfiles\\samples' # sample and output uses same dimensions, so keep this folder clean!
image_list = getFileList(dir_name)

In [4]:
image_list

['mdpfiles\\samples\\1.jpg',
 'mdpfiles\\samples\\2.jpg',
 'mdpfiles\\samples\\3.jpg',
 'mdpfiles\\samples\\4.jpg',
 'mdpfiles\\samples\\5.jpg',
 'mdpfiles\\samples\\6.jpg']

In [5]:
'''
Declare columns for image_df
'''
columns_image = [
    'Image Width',
    'Image Height'
]

image_df = pd.DataFrame(columns = columns_image)

In [6]:
image_df

,Image Width,Image Height


In [7]:
from PIL import Image

'''
For loop to iterate through image paths in image_list
Then, run Image function to get width and height for each image
'''
for image_path in image_list: # I need to load each image name into a list
    im = Image.open(image_path)
    img_width = im.size[0]
    img_height = im.size[1]
    image_df.loc[len(image_df)]=[img_width, img_height]

In [8]:
'''
Combine original df with image_df to store all meta info together
'''
final_df = pd.concat([df, image_df], axis=1, sort=False)

In [9]:
'''
Will need to map a dictionary of values to distances (in terms of how many blocks away)
For now, assume that the blocks away = 0
In future, assume we need to map different bounds for:
0
1
2
3
4
5
6
7
8
9
''' 

final_df['Left Bound'] = (final_df['Image Width']/3) * 1 
final_df['Center Bound'] = (final_df['Image Width']/3) * 2
final_df['Right Bound'] = final_df['Image Width']
final_df['Offset Factor'] = 0

In [10]:
'''
To determine most realistic partition for bounding box

For Left Bounding box
Img X-axis + bounding box length

For Right Bounding box
Img X-axis 

But for now do simplest implementation without bounding box factor

'''

'\nTo determine most realistic partition for bounding box\n\nFor Left Bounding box\nImg X-axis + bounding box length\n\nFor Right Bounding box\nImg X-axis \n\nBut for now do simplest implementation without bounding box factor\n\n'

In [11]:
offset_list = []

for index, row in final_df.iterrows():
    if (row['BBox X-axis'] < row['Left Bound']):
        offset_list.append('Left') 
    elif (row['BBox X-axis'] > row['Center Bound']): 
        offset_list.append('Right')
    else:
        offset_list.append('Center')
        
final_df['Offset Factor'] = offset_list
final_df['Robot X-axis'] = 1
final_df['Robot Y-axis'] = 0
final_df['Robot Orientation'] = None
final_df['Actual Image ID'] = 0
final_df['Actual Image X-axis'] = 0
final_df['Actual Image Y-axis'] = 0

In [13]:
# map description to actual id using list mapping

id_dict = {
    '[ID: 6] six': 6,
    '[ID: 7] seven' : 7,
    '[ID: 8] eight' : 8,
    '[ID: 9] nine' : 9,
    '[ID: 10] zero' : 10,
    '[ID: 11] Alphabet V' : 11,
    '[ID: 12] Alphabet W' : 12,
    '[ID: 13] Alphabet X' : 13,
    '[ID: 14] Alphabet Y' : 14,
    '[ID: 15] Alphabet Z' : 15,
    '[ID: 3] right arrow' : 3,
    '[ID: 5] Go' : 5,
    '[ID: 4] left arrow' : 4,
    '[ID: 2] down arrow' : 2,
    '[ID: 1] Up arrow' : 1
}

final_df['Actual Image ID'] = final_df['Description'].map(id_dict) 

In [14]:
'''
Create output_df and convert to string for sending to Android
'''
output_df = final_df[['Actual Image ID', 'Actual Image X-axis', 'Actual Image Y-axis']]
output_string = output_df.to_string(header=False, index=False)
print(output_string)

  6  0  0
  6  0  0
  6  0  0
 10  0  0
 10  0  0
 10  0  0


In [15]:
'''
Communication function
'''

from img_comm_get import *

print("main")
rec = RecAPI()
rec.init_pc_comm()

'''
rec.read_from_RPI("test.jpg")
print("Image is saved")
'''

send_msg = output_string
print("write_to_Android(): ", send_msg)
rec.write_to_Android(send_msg.encode())

print("closing sockets")
rec.close_pc_socket()

main
Error:  [WinError 10049] The requested address is not valid in its context
Try again in a few seconds
write_to_Android():    6  0  0
  6  0  0
  6  0  0
 10  0  0
 10  0  0
 10  0  0


AttributeError: 'NoneType' object has no attribute 'sendto'